hack_eeg_emotion.py
Dataset: https://springernature.figshare.com/articles/dataset/Preprocessed_Dataset/25751097?backTo=/collections/EEG_Dataset_for_the_Recognition_of_Different_Emotions_Naturally_Induced_in_Voice-User_Interaction/7207839 
Pipeline: band-power → SVM → play tune
Python ≥3.10, needs: numpy, scipy, scikit-learn, playsound

In [1]:
# !pip install numpy #>=1.22
# !pip install scipy #>=1.8
# !pip install scikit-learn #>=1.1
# !pip install playsound #==1.3.0

In [2]:
from pathlib import Path
import pickle, random, os
import numpy as np
from scipy.signal import welch
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from playsound import playsound               # pip install playsound==1.3.0

In [ ]:
DATA_DIR =Path("data/deap")
MUSIC_DIR = Path("music")
FS        = 128                                # Hz, fixed in DEAP

In [ ]:
# ---------- helpers ----------
def bandpower(signal: np.ndarray, band: tuple[int,int], fs: int = FS) -> float:
    f, pxx = welch(signal, fs=fs, nperseg=256)
    idx    = (f >= band[0]) & (f <= band[1])
    return pxx[idx].mean()

BANDS = {"alpha": (8,13), "beta": (13,30), "gamma": (30,45)}

def extract_feats(trial: np.ndarray) -> np.ndarray:
    """trial shape: (40 channels, 8064 samples) → 3-D feature vector"""
    return np.array([
        np.mean([bandpower(ch, BANDS[b]) for ch in trial])
        for b in BANDS
    ])

def emo_label(val: float, ar: float) -> str:
    if val >= 5 and ar >= 5:  return "happy"
    if val >= 5 and ar <  5:  return "relaxed"
    if val <  5 and ar <  5:  return "sad"
    return None     # ignore “low-valence/high-arousal” corner (≈25 % of trials)

In [ ]:
# ---------- load ----------
X, y = [], []
for dat_file in sorted(DATA_DIR.glob("s*.dat")):        # s01.dat … s32.dat
    with dat_file.open("rb") as f:
        obj = pickle.load(f, encoding="latin1")         # {'data','labels'}
    trials, labels = obj["data"], obj["labels"]         # (40,40,8064), (40,4)

    for trial, (val, ar, *_ ) in zip(trials, labels):
        lbl = emo_label(val, ar)
        if lbl is None:                 # skip anxious corner
            continue
        X.append(extract_feats(trial))
        y.append(lbl)

X, y = np.vstack(X), np.array(y)
print(f"Feature matrix {X.shape}, labels {np.unique(y, return_counts=True)}")

In [ ]:
# ---------- model ----------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=42)
clf = SVC(kernel="linear")
clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

In [ ]:
# ---------- demo inference & music ----------
rnd_idx = random.randrange(len(X_test))
pred    = clf.predict([X_test[rnd_idx]])[0]
track   = MUSIC_DIR / f"{pred}.mp3"
print(f"Predicted mood ⇒ {pred}.  Playing {track} …")
if track.exists():
    playsound(track.as_posix())
else:
    print("Missing audio file – drop an mp3 with the right name into ./music/")